<a href="https://colab.research.google.com/github/ignacio727/03MAIR-Algoritmos-de-Optimizacion-2021/blob/main/03MAIR-Algoritmos%20de%20optimizacion/AG3/Ignacio_de_Jaime_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3 de Algoritmos de Optimización

### Ignacio de Jaime Hernández

https://colab.research.google.com/drive/1sDEPYnNizXQ0Ep5QTEKr1tynpxNLvipO?usp=sharing

https://github.com/ignacio727/03MAIR-Algoritmos-de-Optimizacion-2021/tree/main/03MAIR-Algoritmos%20de%20optimizacion/AG3

# El problema del agente viajero - TSP

In [ ]:
!pip install requests
!pip install tsplib95

In [ ]:
import urllib.request
import copy

# Descargar el juego de datos
file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

# Módulos de TSP
import tsplib95
import random
import math

# Cargar datos del problema
problem = tsplib95.load_problem(file)

# Nodos
nodos = list(problem.get_nodes())

# Aristas
list(problem.get_edges())

# Funciones auxiliares

# Genera una solución aleatoria con comienzo en el nodo 0
def crear_solucion(nodos):
  sol = [nodos[0]]
  for n in nodos[1:]:
    sol += [random.choice(list(set(nodos) - set({nodos[0]}) - set(sol)))]
  return sol

# Distancia entre dos nodos
def dist(a, b, problem):
  return problem.get_weight(a, b)

# Distancia total de una trayectoria/solución
def dist_total(sol, problem):
  dist_total = 0
  for i in range(len(sol)-1):
    dist_total += dist(sol[i], sol[i+1], problem)
  return dist_total + dist(sol[len(sol)-1], sol[0], problem)

### Busqueda aleatoria
def busqueda_aleatoria(problem, N):
  nodos = list(problem.get_nodes())

  mejor_sol = []
  mejor_dist = 10e100

  for i in range(N):
    sol = crear_solucion(nodos)
    dist = dist_total(sol, problem)

    if dist < mejor_dist:
      mejor_sol = sol
      mejor_dist = dist

  print('Busqueda aleatoria')
  print('Mejor solución = ', mejor_sol)
  print('Distancia      = ', mejor_dist)
  return mejor_sol

sol = busqueda_aleatoria(problem, 5000)

### Busqueda local
def genera_vecina(sol):
  mejor_sol = []
  mejor_dist = 10e100

  for i in range(1, len(sol)-1):
    for j in range(i+1, len(sol)):
      vecina = sol[:i] + [sol[j]] + sol[i+1:j] + [sol[i]] + sol[j+1:]
      dist_vecina = dist_total(vecina, problem)

      if dist_vecina <= mejor_dist:
        mejor_dist = dist_vecina
        mejor_sol = vecina
  return mejor_sol

def busqueda_local(problem):
  mejor_sol = []

  sol_refer = crear_solucion(nodos)
  mejor_dist = 10e100

  iter = 0
  while(1):
    iter += 1
    vecina = genera_vecina(sol_refer)
    dist_vecina = dist_total(vecina, problem)

    if dist_vecina < mejor_dist:
      mejor_sol = copy.deepcopy(vecina)
      mejor_dist = dist_vecina
    else:
      print('Busqueda local')
      print('En la iteración ', iter, ', la mejor solución encontrada es:', mejor_sol)
      print('Distancia    :', mejor_dist)
      return mejor_sol
    
    sol_refer = vecina

sol2 = busqueda_local(problem)

### Recocido simulado
def genera_vecina_aleatorio(sol):
  i,j = sorted(random.sample(range(1, len(sol)), 2))
  return sol[:i] + [sol[j]] + sol[i+1:j] + [sol[i]] + sol[j+1:]

# Función de probabilidad para determinar si se cambia a una
# solución peor respecto a la de referencia(exponencial)
def probabilidad(T, d):
  if random.random() <= math.exp(-1*d / T):
    return True
  else:
    return False

def bajar_temperatura(T):
  return 0.999*T

def recocido_simulado(problem, T=1000):
  sol_refer = crear_solucion(nodos)
  dist_refer = dist_total(sol_refer, problem)

  mejor_sol = []
  mejor_dist = 10e100

  iter=0
  while T > 1e-4:
    iter += 1
    # Solución vecina
    vecina = genera_vecina_aleatorio(sol_refer)

    # Cálcula su distancia
    dist_vecina = dist_total(vecina, problem)

    # Si es la mejor solución se guarda
    if dist_vecina < mejor_dist:
      mejor_sol = vecina
      mejor_dist = dist_vecina

    # Si la nueva vecina es mejor se cambia
    # Si es peor se cambia según una probabilidad que depende de T y delta
    if dist_vecina < dist_refer or probabilidad(T, abs(dist_refer - dist_vecina)):
      sol_refer = copy.deepcopy(vecina)
      dist_refer = dist_vecina

    # Se baja la temperatura
    T = bajar_temperatura(T)

  print('Recocido simulado')
  print('La mejor solución encontrada es ', mejor_sol)
  print('La distancia total es ', mejor_dist)
  return mejor_sol

sol3 = recocido_simulado(problem, 1e6)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated function (or staticmethod) load_problem. (Will be removed in newer versions. Use `tsplib95.load` instead.) -- Deprecated since version 7.0.0.
  


Busqueda aleatoria
Mejor solución =  [0, 1, 33, 31, 38, 20, 35, 4, 12, 32, 19, 26, 13, 5, 30, 25, 17, 28, 14, 6, 36, 15, 9, 24, 22, 8, 40, 21, 18, 11, 3, 29, 10, 37, 16, 34, 41, 23, 39, 27, 2, 7]
Distancia      =  3845
Busqueda local
En la iteración  35 , la mejor solución encontrada es: [0, 30, 38, 22, 39, 21, 23, 41, 10, 25, 11, 12, 18, 4, 3, 1, 6, 14, 16, 5, 40, 24, 33, 20, 34, 32, 7, 17, 31, 35, 36, 37, 15, 19, 13, 26, 8, 9, 29, 28, 2, 27]
Distancia    : 1901
Recocido simulado
La mejor solución encontrada es  [0, 2, 39, 21, 40, 24, 22, 38, 34, 33, 35, 36, 15, 16, 14, 19, 13, 11, 25, 41, 23, 9, 29, 30, 28, 8, 10, 12, 18, 1, 7, 32, 20, 31, 17, 37, 5, 26, 6, 4, 3, 27]
La distancia total es  1750


In [ ]:
### Método constructivo: Colonia de Hormigas
# Funciones auxiliares
def add_nodo(problem, H, T):
  # Añade un nuevo nodo dependiendo de los nodos más cercanos
  # y de las feromonas depositadas
  nodos = list(problem.get_nodes())
  return random.choice(list(set(range(1, len(nodos))) - set(H)))

def incrementa_feromona(problem, T, H):
  # Incrementa según la calidad de la solución. Añade una cantidad 
  # inversamente proporcional a la distancia total
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/dist_total(H, problem)
  return T

def evaporar_feromonas(T):
  # Evapora 0.3 del valor de la feromona, sin que baje de 1
  T = [[max(T[i][j]-0.3, 1) for i in range(len(nodos))] for j in range(len(nodos))]
  return T

# Función principal
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = add_nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = incrementa_feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = evaporar_feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_sol = []
  mejor_dist = 10e100
  for h in range(N) :
    dist_actual = dist_total(Hormiga[h], problem)
    if dist_actual < mejor_dist:
      mejor_sol = Hormiga[h]
      mejor_dist =dist_actual
  
  
  print(mejor_sol)
  print(mejor_dist)
  
  
hormigas(problem, 1000)

[0, 25, 23, 1, 5, 33, 27, 22, 24, 29, 11, 41, 12, 40, 35, 15, 14, 7, 18, 9, 8, 30, 16, 17, 37, 36, 20, 38, 19, 26, 34, 21, 6, 4, 2, 3, 31, 13, 10, 39, 28, 32]
3946
